In [1]:
import pulp
import sys
sys.path.insert(0, '../')
import polars as pl
import numpy as np
from src.data_synthesizer import DataSynthesizer

In [2]:
pl.Config(tbl_width_chars=1000)

In [3]:
# Load sample data using DataSynthesizer 

ds = DataSynthesizer(10000, 3000) # e.g., 10000 trips, 3000 unsuccesful trips
trip_df = ds.trip_hist_data
charge_df = ds.charge_status_data
car_q_pop_df = ds.car_q_pop_data
park_q_pop_df = ds.park_q_pop_data

In [4]:
print(trip_df)

shape: (3_000, 7)
┌────────────────────────────┬────────────────────────────┬──────────────────┬────────────────┬────────────────────┬─────────┬─────────────┐
│ trip_start_datetime        ┆ trip_end_datetime          ┆ start_station_id ┆ end_station_id ┆ distance_travelled ┆ car_id  ┆ customer_id │
│ ---                        ┆ ---                        ┆ ---              ┆ ---            ┆ ---                ┆ ---     ┆ ---         │
│ datetime[μs]               ┆ datetime[μs]               ┆ i64              ┆ i64            ┆ f64                ┆ str     ┆ str         │
╞════════════════════════════╪════════════════════════════╪══════════════════╪════════════════╪════════════════════╪═════════╪═════════════╡
│ 2024-04-29 23:58:58.426276 ┆ 2024-04-30 00:58:42.426276 ┆ 141              ┆ 343            ┆ 39.205389          ┆ CAR2563 ┆ CUS4759     │
│ 2023-08-14 00:40:06.426298 ┆ 2023-08-14 02:20:45.426298 ┆ 54               ┆ 347            ┆ 18.518535          ┆ CAR2127 ┆ CUS3108  

In [5]:
print(charge_df)

shape: (3_000, 7)
┌────────────────────────────┬────────────────────────────┬─────────────────┬──────────────────┬─────────┬───────────────────────────┬────────────────────────────┐
│ left_station_datetime      ┆ enter_station_datetime     ┆ left_station_id ┆ enter_station_id ┆ car_id  ┆ left_station_charge_level ┆ enter_station_charge_level │
│ ---                        ┆ ---                        ┆ ---             ┆ ---              ┆ ---     ┆ ---                       ┆ ---                        │
│ datetime[μs]               ┆ datetime[μs]               ┆ i64             ┆ i64              ┆ str     ┆ i64                       ┆ i64                        │
╞════════════════════════════╪════════════════════════════╪═════════════════╪══════════════════╪═════════╪═══════════════════════════╪════════════════════════════╡
│ 2024-04-29 23:58:58.426276 ┆ 2024-04-30 00:58:42.426276 ┆ 141             ┆ 343              ┆ CAR2563 ┆ 20                        ┆ 7                          

In [6]:
print(car_q_pop_df)

shape: (4_000, 5)
┌────────────────────────────┬────────────────────────────┬──────────────────┬──────────────┬─────────────┐
│ event_creation_datetime    ┆ event_expiry_datetime      ┆ start_station_id ┆ event_status ┆ customer_id │
│ ---                        ┆ ---                        ┆ ---              ┆ ---          ┆ ---         │
│ datetime[μs]               ┆ datetime[μs]               ┆ i64              ┆ str          ┆ str         │
╞════════════════════════════╪════════════════════════════╪══════════════════╪══════════════╪═════════════╡
│ 2024-04-29 23:30:58.426276 ┆ 2024-04-30 00:00:58.426276 ┆ 141              ┆ successful   ┆ CUS4759     │
│ 2023-08-14 00:12:06.426298 ┆ 2023-08-14 00:42:06.426298 ┆ 54               ┆ successful   ┆ CUS3108     │
│ 2023-12-09 10:03:12.426304 ┆ 2023-12-09 10:33:12.426304 ┆ 297              ┆ successful   ┆ CUS1132     │
│ 2024-04-29 11:20:03.426309 ┆ 2024-04-29 11:50:03.426309 ┆ 304              ┆ successful   ┆ CUS3509     │
│ 2023-10-

In [7]:
print(park_q_pop_df)

shape: (4_000, 5)
┌────────────────────────────┬────────────────────────────┬────────────────┬──────────────┬─────────────┐
│ event_creation_datetime    ┆ event_expiry_datetime      ┆ end_station_id ┆ event_status ┆ customer_id │
│ ---                        ┆ ---                        ┆ ---            ┆ ---          ┆ ---         │
│ datetime[μs]               ┆ datetime[μs]               ┆ i64            ┆ str          ┆ str         │
╞════════════════════════════╪════════════════════════════╪════════════════╪══════════════╪═════════════╡
│ 2024-04-30 00:00:58.426276 ┆ 2024-04-30 00:10:58.426276 ┆ 343            ┆ successful   ┆ CUS4759     │
│ 2023-08-14 00:42:06.426298 ┆ 2023-08-14 00:52:06.426298 ┆ 347            ┆ successful   ┆ CUS3108     │
│ 2023-12-09 10:33:12.426304 ┆ 2023-12-09 10:43:12.426304 ┆ 158            ┆ successful   ┆ CUS1132     │
│ 2024-04-29 11:50:03.426309 ┆ 2024-04-29 12:00:03.426309 ┆ 26             ┆ successful   ┆ CUS3509     │
│ 2023-10-22 23:04:43.426314

In [23]:
import polars as pl


num_stations = 380
periods = 5 # assume we divide the day into 5 periods
np.random.seed(42)

station_ids = [f'Station_{i+1}' for i in range(num_stations)]
time_periods = list(range(periods))

# Generate unmet car reservation demand and unmet parking reservation demand for all stations and periods
unmet_car_demand = np.random.randint(0, 4, size=(num_stations, periods))
unmet_parking_demand = np.random.randint(0, 4, size=(num_stations, periods))

failed_car_reservations = {(station_ids[i], t): unmet_car_demand[i, t] for i in range(num_stations) for t in time_periods}
failed_parking_reservations = {(station_ids[i], t): unmet_parking_demand[i, t] for i in range(num_stations) for t in time_periods}

predicted_demand = {(station_ids[i], t): unmet_car_demand[i, t] + unmet_parking_demand[i, t] + np.random.randint(1, 5)
                    for i in range(num_stations) for t in time_periods}

# Create a list for each column
stations = [station_ids[i] for i in range(num_stations) for t in time_periods]
time_periods_list = [t for i in range(num_stations) for t in time_periods]
failed_car_reservations_list = [failed_car_reservations[(station_ids[i], t)] for i in range(num_stations) for t in time_periods]
failed_parking_reservations_list = [failed_parking_reservations[(station_ids[i], t)] for i in range(num_stations) for t in time_periods]
predicted_demand_list = [predicted_demand[(station_ids[i], t)] for i in range(num_stations) for t in time_periods]

# Create a DataFrame from the lists
df = pl.DataFrame({
    'Station': stations,
    'Time Period': time_periods_list,
    'Failed Car Reservations': failed_car_reservations_list,
    'Failed Parking Reservations': failed_parking_reservations_list,
    'Predicted or Average Demand': predicted_demand_list
})

In [24]:
print(df)

shape: (1_900, 5)
┌─────────────┬─────────────┬─────────────────────────┬─────────────────────────────┬─────────────────────────────┐
│ Station     ┆ Time Period ┆ Failed Car Reservations ┆ Failed Parking Reservations ┆ Predicted or Average Demand │
│ ---         ┆ ---         ┆ ---                     ┆ ---                         ┆ ---                         │
│ str         ┆ i64         ┆ i64                     ┆ i64                         ┆ i64                         │
╞═════════════╪═════════════╪═════════════════════════╪═════════════════════════════╪═════════════════════════════╡
│ Station_1   ┆ 0           ┆ 2                       ┆ 3                           ┆ 9                           │
│ Station_1   ┆ 1           ┆ 3                       ┆ 0                           ┆ 7                           │
│ Station_1   ┆ 2           ┆ 0                       ┆ 0                           ┆ 2                           │
│ Station_1   ┆ 3           ┆ 2                       

In [25]:
prob = pulp.LpProblem("EVChargingPortExpansion", pulp.LpMinimize)

In [26]:
additional_ports = pulp.LpVariable.dicts("AdditionalPorts", station_ids, lowBound=0, cat='Integer')

In [29]:
D_c = 1  # Dissatisfaction cost per failed car reservation
D_p = 1  # Dissatisfaction cost per failed parking reservation
prob += pulp.lpSum(D_c * failed_car_reservations[(s, t)] + D_p * failed_parking_reservations[(s, t)]
                   for s in station_ids for t in time_periods)

In [32]:
current_capacity = {station: 4 for station in station_ids}

In [44]:
installation_cost_per_port = 1000  # Cost per additional port
budget = 9999999999999999  # Total budget (you may want to adjust this based on the scale)

In [52]:
# Constraints
for s in station_ids:
    # New capacity must meet predicted demand
    for t in time_periods:
        prob += current_capacity[s] + additional_ports[s] >= predicted_demand[(s, t)]

# Budget constraint
prob += pulp.lpSum(additional_ports[s] * installation_cost_per_port for s in station_ids) <= budget

# Solve the problem
prob.solve()


results = {s: pulp.value(additional_ports[s]) for s in station_ids if pulp.value(additional_ports[s]) > 0}
results_df = pl.DataFrame(list(results.items()), ["Station", "Additional Ports"])
print(results_df.sort('Additional Ports').reverse())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/codespace/.cache/pypoetry/virtualenvs/car-sharing-case-study-2W5MY4dm-py3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a40a3d8e354b4c37822f2c180d5f179c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/a40a3d8e354b4c37822f2c180d5f179c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1364 COLUMNS
At line 3266 RHS
At line 4626 BOUNDS
At line 4678 ENDATA
Problem MODEL has 1359 rows, 51 columns and 1800 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding ro